In [7]:
from sagas.ofbiz.builder import oc, finder, abbrev, get_graphql_type, get_mapping_type
from sagas.util.str_converters import to_camel_case, to_snake_case
print(oc.delegator.getDelegatorName())

default


In [13]:
entities=['TestingType']
mutations_c=[]
mutations_c.append("class Mutations(graphene.ObjectType):")

entity_name='TestingType'
gen_ctx={}    
gen_ctx['model']=entity_name
gen_ctx['s_model']=to_snake_case(entity_name)
ent = oc.delegator.getModelEntity(entity_name)
autoflds=ent.getAutomaticFieldNames()
    
model_c=[]
model_c.append("class {ctx[model]}Input(graphene.InputObjectType):".format(ctx=gen_ctx))

names = ent.getAllFieldNames()
for field_name in names:
    if field_name not in autoflds:
        fld = ent.getField(field_name)
        fldtype = get_graphql_type(fld.getType())
        model_c.append("    {name} = graphene.{type}()".format(name=to_snake_case(field_name),
                                                                 type=fldtype))
print("\n".join(model_c))

class TestingTypeInput(graphene.InputObjectType):
    testing_type_id = graphene.String()
    description = graphene.String()


In [15]:
creator_def='''
class Create{ctx[model]}(graphene.Mutation):
    class Arguments:
        {ctx[s_model]}_data = {ctx[model]}Input(required=True)

    {ctx[s_model]} = graphene.Field(lambda: {ctx[model]})
    Output = {ctx[model]}

    @staticmethod
    def mutate(root, info, {ctx[s_model]}_data=None):
        {ctx[s_model]} = input_to_dictionary({ctx[s_model]}_data, "{ctx[model]}", {ctx[model]})        
        return {ctx[s_model]}
'''
model_c.append(creator_def.format(ctx=gen_ctx))
mutations_c.append("    create_{ctx[s_model]} = Create{ctx[model]}.Field()".format(ctx=gen_ctx))
print("\n".join(model_c))

class TestingTypeInput(graphene.InputObjectType):
    testing_type_id = graphene.String()
    description = graphene.String()

class CreateTestingType(graphene.Mutation):
    class Arguments:
        testing_type_data = TestingTypeInput(required=True)

    testing_type = graphene.Field(lambda: TestingType)
    Output = TestingType

    @staticmethod
    def mutate(root, info, testing_type_data=None):
        testing_type = input_to_dictionary(testing_type_data, "TestingType", TestingType)        
        return testing_type



In [16]:
print("\n".join(model_c))
print("\n".join(mutations_c))

class TestingTypeInput(graphene.InputObjectType):
    testing_type_id = graphene.String()
    description = graphene.String()

class CreateTestingType(graphene.Mutation):
    class Arguments:
        testing_type_data = TestingTypeInput(required=True)

    testing_type = graphene.Field(lambda: TestingType)
    Output = TestingType

    @staticmethod
    def mutate(root, info, testing_type_data=None):
        testing_type = input_to_dictionary(testing_type_data, "TestingType", TestingType)        
        return testing_type

class Mutations(graphene.ObjectType):
    create_testing_type = CreateTestingType.Field()
